In [1]:
import numpy as np
import h5py
import pickle
from tqdm import tqdm
import sys

from scipy.constants import physical_constants
kB = physical_constants["Boltzmann constant in eV/K"][0]

In [2]:
# Let's load the data first
T = 1073
start = 16000
end = 18000
intervalPath = "1073_step1/16e3_18e3/"
fl_dat = intervalPath + "data_{}_0_{}.h5".format(T, start)
Init_States = np.load("states_{}_0.npy".format(T))
with h5py.File(fl_dat, "r") as fl:
    FinalStates = np.array(fl["FinalStates"])
    SpecDisps = np.array(fl["SpecDisps"])
    tarr = np.array(fl["times"])
    JumpSelects = np.array(fl["JumpSelects"])
    TestRandomNums = np.array(fl["TestRandNums"])
    TestRates = np.array(fl["TestRates"])
    TestBarriers = np.array(fl["TestBarriers"])

In [4]:
# Next we need the neighborhood information
NNsites = np.load("../CrysDat_FCC/NNsites_sitewise.npy")
NNsites_vac = NNsites[1:, 0]
NNsites_vac

array([ 15,  57, 449,  71, 448,  64,   7,   1,   8,  56, 456, 120])

In [5]:
# next, get the displacements
dxJumps = np.load("../CrysDat_FCC/dxList.npy") * 3.59
dxJumps

array([[ 0.   , -1.795,  1.795],
       [-0.   ,  1.795, -1.795],
       [ 1.795,  0.   , -1.795],
       [-1.795, -0.   ,  1.795],
       [ 0.   , -1.795, -1.795],
       [-0.   ,  1.795,  1.795],
       [-1.795, -1.795,  0.   ],
       [ 1.795,  1.795, -0.   ],
       [ 1.795,  0.   ,  1.795],
       [-1.795, -0.   , -1.795],
       [ 1.795, -1.795,  0.   ],
       [-1.795,  1.795, -0.   ]])

In [6]:
# First, let's check that the vacancies have been put in properly
assert np.all(FinalStates[np.arange(FinalStates.shape[0]), NNsites_vac[JumpSelects]] == 0)

# Next, check that the correct displacements have been stored for the correct species
z = np.zeros(3)
for traj in tqdm(range(2000), position=0, leave=True):
    state1 = Init_States[traj + start, :]
    state2 = FinalStates[traj, :]
    
    SpecJump = state1[NNsites_vac[JumpSelects[traj]]]
    assert state2[0] == SpecJump
    assert state1[0] == 0
    
    # Now check the displacements
    assert np.allclose(np.sum(SpecDisps[traj], axis=0), z)
    assert np.allclose(SpecDisps[traj, 0], dxJumps[JumpSelects[traj]])
    
    for sp in range(1, 6):
        if sp == SpecJump:
            assert np.allclose(SpecDisps[traj, sp], -dxJumps[JumpSelects[traj]])
        else:
            assert np.allclose(SpecDisps[traj, sp], z)

100%|█████████████████████████████████████| 2000/2000 [00:00<00:00, 2223.17it/s]


In [7]:
# Check lammps input writing
specs = np.zeros_like(Init_States[0])
with open(intervalPath + "initial_19.data", "r") as fl:
    lines = fl.readlines()
    atomLines = lines[12:]
    for atom in atomLines:
        splitLine = atom.split()
        idx = int(splitLine[0])
        sp = int(splitLine[1])
        specs[idx] = sp
assert np.all(Init_States[end-1] == specs)

In [8]:
# Check lammps output writing
batchSize = 20
for traj in range(batchSize):
    for jumpInd in range(NNsites_vac.shape[0]):
        with open(intervalPath+"final_{}_{}.data".format(traj, jumpInd)) as fl:
            lines = fl.readlines()
            atomLines = lines[1:]
        site = NNsites_vac[jumpInd] - 1
        coords = np.array([float(x) for x in atomLines[site].split()[1:]])
        assert np.allclose(coords, z)

In [9]:
# Now check the stochastic decisions
for samp in range(batchSize):
    Barriers = TestBarriers[samp]
    rates = TestRates[samp]
    rn = TestRandomNums[samp]
    assert np.allclose(rates, np.exp(-Barriers/(kB*T)))
    
    rateSum = np.sum(rates)
    assert np.math.isclose(tarr[samp], 1.0/rateSum)
    
    rateProbs = np.cumsum(rates/rateSum)
    assert JumpSelects[samp] == np.searchsorted(rateProbs, rn)